<a href="https://colab.research.google.com/github/Lorxus/Project-Newcomb/blob/main/NimRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# defines <=4-column Nim
import numpy as np
import random as rand

# defines game
class Nim:
    def __init__(self, board = [0, 0, 0, 0], firstplayerhuman = False, secondplayerhuman = False):  # 4 colors of berry, why not
        self.board = board
        self.startboard = board

        self.players = ['1', '2']

        tempplayers = []
        if firstplayerhuman:
            tempplayers.append('1')
        if secondplayerhuman:
            tempplayers.append('2')
        self.humans = tempplayers

        self.current_player = '1'
        self.winner = None
        self.game_over = False
        self.reward: float | None = None

    def reset(self):
        self.board = self.startboard
        self.current_player = '1'
        self.winner = None
        self.game_over = False

    def get_available_moves(self):  # pick a pile and eat any number of the same color
        moves = []
        for i in range(4):
            for j in range(self.board[i-1]):
              moves.append((i, j))
        return moves

    def make_move(self, move):  # this isn't going to be how human players submit moves, just backend
        if move == None or self.currentplayer in self.humans:  # we'll use a dummy move for the human player...
            move = self.prompthumanmove()  # ...then overwrite it

        self.board[move[0]] = self.board[move[0]] - self.board[move[1]]  # the human player can't submit illegal moves and the CPU never will

        if self.check_winner() is not None:  # if the game isn't over, switch players
            self.switch_player()
            self.print_board()
            return True
        else:  # if it is, then say so
            print('Game over! Winner: Player', self.current_player)
            self.score()

    def switch_player(self):
        if self.current_player == self.players[0]:
          self.current_player = self.players[1]
        else:
          self.current_player = self.players[0]

    def check_winner(self): # if there's no berries left at the end of your turn you win
        if sum(self.board) > 0:
          return None
        elif self.current_player == '1':
          return '1'
        else:
          return '2'

    def get_board(self):
        return self.board

    def print_board(self):
        humanity = self.currentplayer in self.humans
        nottedness = 'is'
        if not humanity:
            nottedness = 'is not'
        print('Current player is: Player', self.currentplayer, 'who', nottedness, 'a human player.', 'The board state is:', self.board[0], 'red,', self.board[1], 'yellow,', self.board[2], 'green,', self.board[3], 'blue.')
        return True

    def score(self):
        if self.game_over == False:
          self.reward = None
        elif self.winner == '1':
          self.reward = 1
        elif self.winner == '2':
          self.reward = -1

    def prompthumanmove(self):
        print('Enter pile number (0-3).')
        pilenumber = int(input())
        while self.board[pilenumber] < 1:
            print('Pile is empty - pick a different one?')
            pilenumber = int(input)

        print('Pile', pilenumber, 'has', self.board[pilenumber], 'berries; please enter that number or fewer to make your move.')
        berrynumber = int(input)
        while berrynumber > self.board[pilenumber] or berrynumber < 1:
            print('Bad number - try that again?')
            berrynumber = int(input)

        mymove = [pilenumber, berrynumber]
        return mymove

    def testcase_factory(param: int):  # creates a random board with max berries param per pile and returns the Nim game that has that board as its start state
        randboard = []
        for i in range(4):
            temprand = rand.randint(0, param)
            randboard.append(temprand)

        newgame = Nim(randboard)
        return newgame

# need to code up a playable test case for two human players...

In [ ]:
# implement Q-learning
# this is the part that actually invokes Q-learning/RL
class QLearningAgent:
  def __init__(self, alpha, epsilon, discount_factor):
    self.Q = {}  # (state, action). what have we tried so far?
    self.payoff = {}  # ((state, action), payoff). what did it result in?/what is it predicted/expected to result it?
    self.alpha = alpha  # learning rate
    self.epsilon = epsilon  # exploration rate
    self.gamma = discount_factor  # time-discount factor

  # build up a dictionary of state-action pairs with associated node values (Q-values) - this part will need to interface very directly with any IB stuff I do
  def get_Q_value(self, state, action):
    if (state, action) not in self.Q:
      self.Q[(state, action)] = 0.0
    return self.Q[(state, action)]

  # update the Q-values labelling the entries based on difference between expected and actual rewards
  # this will also need to be heavily adapted for IB, if/when I get there
  def update_Q_value(self, state, action, next_state):
    oldQ = self.get_Q_value(state, action)

    Q_values = [self.get_Q_value(next_state, action) for action in Nim(state).get_available_moves()]
    max_Q = max(Q_values)

    self.Q[(state, action)] = oldQ + self.alpha * ((self.gamma *  max_Q) - oldQ)  # immediate reward for Nim is always 0
    # Q(S_t, A_t) = Q(S_t, A_t) + a(R_t+1 + g max_act Q(S_t+1, A_t+1) - Q(S_t, A_t))

  # choose an action
  # with p=eps, pick a random new one - this might look like it'll cause problems with IB but actually there's a specific way to get around that.
  # with p = 1-eps, pick an action with the greatest expected node value (Q-value).
  def choose_action(self, state):
    if rand.uniform(0, 1) < self.epsilon:
      return rand.choice(Nim(state).get_available_moves())
    else:
      Q_values = [self.get_Q_value(state, action) for action in Nim(state).get_available_moves()]
      max_Q = max(Q_values)
      if Q_values.count(max_Q) > 1:
        best_moves = [i for i in range(len(Nim(state).get_available_moves())) if Q_values[i] == max_Q]
        i = rand.choice(best_moves)
      else:
        i = Q_values.index(max_Q)
      return Nim(state).get_available_moves()[i]


  # EVERYTHING BELOW IS INCOMPLETE.
  # I will likely need to redo a lot of it!
  # trains creates a RL bot with specified learning rate alpha, exploration rate eps, and time-discount factor discount_factor to train for num_episodes games.
  def train(self, num_episodes):
    agent = QLearningAgent(self.alpha, self.epsilon, self.gamma)
    gameslate = []
    for i in range(num_episodes):
        randomboard = []
        for j in range(4):
            randomboard.append(rand.randint(0, 15))
        game = Nim(randomboard)
        gameslate.append(game)

    for i in range(num_episodes):
        game = gameslate[i]
        state = game.board
        while not game.game_over:
            action = agent.choose_action(state)
            next_state = game.make_move(action)
            agent.update_Q_value(game.board, action, next_state)
            game = next_state
    return agent

  # tests the resulting trained agent
  # this part is confused. it probably needs to pull a slate of Nim games?
  def testwinrate(self, num_games):
    num_wins = 0
    gameslate = []
    for i in range(num_games):
        randomboard = []
        for j in range(4):
            randomboard.append(rand.randint(0, 15))
        game = Nim(randomboard)
        gameslate.append(game)

    for i in range(num_games):
        game = gameslate[i]
        state = game.board
        while not game.game_over:
            if game.current_player == '1':
                action = agent.choose_action(state)
            else:
                action = rand.choice(game.get_available_moves())
            state = game.make_move(action)

        score = game.score()
        if score == 1:
            num_wins += 1
    return num_wins / num_games * 100

# creates a test Nim game with specified piles
boardstate = [2, 1, 0, 0]
game = Nim(boardstate)
agent = QLearningAgent(alpha=0.5, epsilon=0.1, discount_factor=1.0)

# train the Q-learning agent
agent = agent.train(num_episodes=100000)

# test the Q-learning agent
win_percentage = agent.testwinrate(num_games=1000)
print("Win percentage: {:.2f}%".format(win_percentage))

TypeError: unhashable type: 'list'